In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, make_scorer, cohen_kappa_score, roc_auc_score, balanced_accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [4]:
# Binary feather files
train_f = pd.read_feather("../data/binary/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f = pd.read_feather("../data/binary/val_all_features.f", columns=None, use_threads=True, storage_options=None)
train_shape = train_f.shape
val_shape = val_f.shape
print(f'train shape: {train_shape}, val shape: {val_shape}')
# Multi-class feather files.
train_f_mul = pd.read_feather("../data/three_class/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f_mul = pd.read_feather("../data/three_class/val_all_feat.f", columns=None, use_threads=True, storage_options=None)
train_shape_m = train_f_mul.shape
val_shape_m = val_f_mul.shape
print(f'train shape multi: {train_shape_m}, val shape multi: {val_shape_m}')

train shape: (15195, 144), val shape: (3796, 144)
train shape multi: (5082, 144), val shape multi: (1270, 144)


In [5]:
## Pre-processing
# Binary case
train_f.replace({'nevus': 1, 'others': 0}, inplace=True)
val_f.replace({'nevus': 1, 'others': 0}, inplace=True)

df_train = train_f.iloc[:train_shape[0], :].sample(frac=1, random_state=42)
df_val = val_f.iloc[:val_shape[0], :].sample(frac=1, random_state=42)
print(f"df_train: {df_train.shape}, df_val: {df_val.shape}")

X_train, y_train = df_train.iloc[:, :(train_shape[1]-1)].to_numpy(dtype=np.float32), df_train.iloc[:, (train_shape[1]-1)].to_numpy()
X_test, y_test = df_val.iloc[:, :(val_shape[1]-1)].to_numpy(dtype= np.float32), df_val.iloc[:, (val_shape[1]-1)].to_numpy()

print(f'X_train_new: {X_train.dtype}, y_train: {y_train.shape}, X_test_new: {X_test.shape}, y_test: {y_test.shape}')

X_train_f = pd.DataFrame(X_train)
X_test_f = pd.DataFrame(X_test)
y_train_f = pd.DataFrame(y_train)
y_test_f = pd.DataFrame(y_test)
frames = [X_train_f, X_test_f]
f = [y_train_f, y_test_f]
X = pd.concat(frames)
y = pd.concat(f)

# Feature Selection -- VARIANCE THRESHOLD
sel_var = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_var= sel_var.fit_transform(X)
print(f'X_var: {X_var.shape}')

## Feature Selection -- SelectFromModel
sel_sfm = SelectFromModel(LinearSVC(C = 0.09, penalty="l1",  dual=False))
print("Performing Feature Selection ... ")
X_sfm = sel_sfm.fit_transform(X, y)
print(f'X_sfm: {X_sfm.shape}')

# split dataframe X again into train and test -- VARIANCE THRESHOLD
X_train_var = pd.DataFrame(X_var).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_var = pd.DataFrame(X_var).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_var: {X_train_var.shape}, y_train: {y_train.shape}, X_test_var: {X_test_var.shape}, y_test: {y_test.shape}')

# split dataframe X again into train and test SFM
X_train_new = pd.DataFrame(X_sfm).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_new = pd.DataFrame(X_sfm).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_new: {X_train_new.shape}, y_train: {y_train.shape}, X_test_new: {X_test_new.shape}, y_test: {y_test.shape}')

df_train: (15195, 144), df_val: (3796, 144)
X_train_new: float32, y_train: (15195,), X_test_new: (3796, 143), y_test: (3796,)
X_var: (18991, 91)
Performing Feature Selection ... 


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X_sfm: (18991, 87)
X_train_var: (15195, 91), y_train: (15195,), X_test_var: (3796, 91), y_test: (3796,)
X_train_new: (15195, 87), y_train: (15195,), X_test_new: (3796, 87), y_test: (3796,)


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [8]:
# Logistic Regression -- without feature selection
param_grid = {'classifier__C': [0.001, 0.01, 0.1, 1,2,3],
              'classifier__penalty' :["l1","l2","elasticnet","none"]}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = LogisticRegression(random_state=42)
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 2)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.1s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s


INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU


[CV] END ........classifier__C=0.001, classifier__penalty=l1; total time=   0.0s
[CV] END ........classifier__C=0.001, classifier__penalty=l2; total time=   0.1s


INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU


[CV] END ........classifier__C=0.001, classifier__penalty=l2; total time=   0.1s
[CV] END ........classifier__C=0.001, classifier__penalty=l2; total time=   0.1s


...

[CV] END ..........classifier__C=3, classifier__penalty=none; total time=   0.4s


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU
C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If th

[CV] END ..........classifier__C=3, classifier__penalty=none; total time=   0.4s


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
INFO:root:sklearn.linear_model.LogisticRegression.fit: running accelerated version on CPU


TRAIN- The best parameters are {'classifier__C': 3, 'classifier__penalty': 'l2'} with an accuracy of 0.8014


In [9]:
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU
INFO:root:sklearn.linear_model.LogisticRegression.predict: running accelerated version on CPU


F1 Score of train data: 0.8046 Acc: 0.8046067785455742
F1 Score of test data: 0.7992 Acc: 0.7992623814541623
